In [1]:
#Scott Scheraga 

#ASEN 5519 Final Project:
#Wire Routing in 2D and 3D Environments
#Using Multi-level Path Planning
#12/13/2020


#Uncomment ""%matplotlib widget"  and comment out "%matplotlib inline" to enable rotations of 3D plots. 
#Requires jupyter-widgets/jupyterlab-manager, jupyter-matplotlib and ,jupyter-plotly

%matplotlib widget  

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

import matplotlib.tri as mtri
from matplotlib import cm

import time
from datetime import datetime

import numpy as np
from numpy import mean
from scipy.spatial import Delaunay
import polytope as pc 
from polytope import extreme, cheby_ball, bounding_box, is_fulldim
import math
from matplotlib.ticker import (AutoMinorLocator, MultipleLocator)
import random
from random import seed
from random import random
from datetime import datetime
import json



#Comment this this when uncommenting %matplotlib widget
#%matplotlib inline   



`polytope` failed to import `cvxopt.glpk`.
will use `scipy.optimize.linprog`


## rectangle 3d plot

In [2]:
#https://stackoverflow.com/questions/13685386/matplotlib-equal-unit-length-with-equal-aspect-ratio-z-axis-is-not-equal-to
def set_aspect_equal_3d(ax):
    """Fix equal aspect bug for 3D plots."""
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    
    xmean = mean(xlim)
    ymean = mean(ylim)
    zmean = mean(zlim)
    plot_radius = max([abs(lim - mean_)
                       for lims, mean_ in ((xlim, xmean),(ylim, ymean),(zlim, zmean))
                       for lim in lims])
    ax.set_xlim3d([xmean - plot_radius, xmean + plot_radius])
    ax.set_ylim3d([ymean - plot_radius, ymean + plot_radius])
    ax.set_zlim3d([zmean - plot_radius, zmean + plot_radius])

def rectangleplot(xmin,xmax,ymin,ymax,Color,Color2,fill=0,Alpha=0):
    plt.plot([xmin,xmin],[ymin,ymax],Color)
    plt.plot([xmax,xmax],[ymin,ymax],Color)
    plt.plot([xmin,xmax],[ymin,ymin],Color)
    plt.plot([xmin,xmax],[ymax,ymax],Color)
    if fill==1:
        plt.fill_between([xmin,xmax], ymin, ymax, color=Color2, alpha=Alpha)      
    
def rectangle3Dplot(xmin,xmax,ymin,ymax,zmin,zmax,color,lines,solids):
    linewidth=1
    x = [xmax,xmax,xmin,xmin]
    y = [ymax,ymin,ymin,ymax]
    z = [zmin,zmin,zmin,zmin]
    if solids==1:
        ax.add_collection3d(Poly3DCollection([list(zip(x,y,z))],facecolors=color, linewidths=linewidth))
    if lines==1:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
        ax.add_collection3d(Line3DCollection([list(zip(x,y,z))], colors='k', linewidths=linewidth))
    x = [xmax,xmax,xmin,xmin]
    y = [ymax,ymin,ymin,ymax]
    z = [zmax,zmax,zmax,zmax]
    if solids==1:
        ax.add_collection3d(Poly3DCollection([list(zip(x,y,z))],facecolors=color, linewidths=linewidth))
    if lines==1:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
        ax.add_collection3d(Line3DCollection([list(zip(x,y,z))], colors='k', linewidths=linewidth))
    x = [xmax,xmax,xmax,xmax]
    y = [ymax,ymin,ymin,ymax]
    z = [zmin,zmin,zmax,zmax]
    if solids==1:
        ax.add_collection3d(Poly3DCollection([list(zip(x,y,z))],facecolors=color, linewidths=linewidth))
    if lines==1:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
        ax.add_collection3d(Line3DCollection([list(zip(x,y,z))], colors='k', linewidths=linewidth))
    x = [xmin,xmin,xmin,xmin]
    y = [ymin,ymax,ymax,ymin]
    z = [zmin,zmin,zmax,zmax]
    if solids==1:
        ax.add_collection3d(Poly3DCollection([list(zip(x,y,z))],facecolors=color, linewidths=linewidth))
    if lines==1:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
        ax.add_collection3d(Line3DCollection([list(zip(x,y,z))], colors='k', linewidths=linewidth))
    x = [xmax,xmin,xmin,xmax]
    y = [ymax,ymax,ymax,ymax]
    z = [zmin,zmin,zmax,zmax]
    if solids==1:
        ax.add_collection3d(Poly3DCollection([list(zip(x,y,z))],facecolors=color, linewidths=linewidth))
    if lines==1:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
        ax.add_collection3d(Line3DCollection([list(zip(x,y,z))], colors='k', linewidths=linewidth))
    x = [xmax,xmin,xmin,xmax]
    y = [ymin,ymin,ymin,ymin]
    z = [zmin,zmin,zmax,zmax]
    if solids==1:
        ax.add_collection3d(Poly3DCollection([list(zip(x,y,z))],facecolors=color, linewidths=linewidth))
    if lines==1:
        x.append(x[0])
        y.append(y[0])
        z.append(z[0])
        ax.add_collection3d(Line3DCollection([list(zip(x,y,z))], colors='k', linewidths=linewidth))
  

In [3]:
def defineworkspace(workspace):
    if workspace==0:
        wo1 = np.array([[3.5,0.5],[4.5,0.5],[4.5,1.5],[3.5,1.5]])
        wo2 = np.array([[6.5,-1.5],[7.5,-1.5],[7.5,-0.5],[6.5,-0.5]])
        obstacles = [wo1,wo2]   
    if workspace==1:
        wo1 = np.array([[1,1],[2,1],[2,5],[1,5]])
        wo2 = np.array([[3,4],[4,4],[4,12],[3,12]])
        wo3 = np.array([[3,12],[12,12],[12,13],[3,13]])
        wo4 = np.array([[12,5],[13,5],[13,13],[12,13]])
        wo5 = np.array([[6,5],[12,5],[12,6],[6,6]])
        obstacles = [wo1,wo2,wo3,wo4,wo5]
    if workspace==2:
        wo1 = np.array([[-6,-6],[25,-6],[25,-5],[-6,-5]])
        wo2 = np.array([[-6,5],[30,5],[30,6],[-6,6]])
        wo3 = np.array([[-6,-5],[-5,-5],[-5,5],[-6,5]])
        wo4 = np.array([[4,-5],[5,-5],[5,1],[4,1]])
        wo5 = np.array([[9,0],[10,0],[10,5],[9,5]])
        wo6 = np.array([[14,-5],[15,-5],[15,1],[14,1]])
        wo7 = np.array([[19,0],[20,0],[20,5],[19,5]])
        wo8 = np.array([[24,-5],[25,-5],[25,1],[24,1]])
        wo9 = np.array([[29,0],[30,0],[30,5],[29,5]])
        obstacles = [wo1,wo2,wo3,wo4,wo5,wo6,wo7,wo8,wo9]
 
#Obsolete obstacle definitions. I have switched to Min/Max for each coordinate, as opposed to bountary points. 
#     if workspace==3: #final project 2d obstacles
#         wo1 = np.array([[5.315,4.921],[5.315,4.803],[4.134,4.803],[4.134,4.921]])
#         wo2 = np.array([[5.315,3.937],[5.315,3.819],[4.134,3.819],[4.134,3.937]])
#         wo3 = np.array([[1.378,3.74],[2.559,3.74],[2.559,3.622],[1.378,3.622]]) 
#         wo4 = np.array([[1.378,2.756],[2.559,2.756],[2.559,2.638],[1.378,2.638]])  
#         wo5 = np.array([[1.97,1.801],[1.772,1.689],[1.772,1.463],
#                         [1.97,1.348],[2.165,1.463],[2.165,1.689]]) 
#         wo6 = np.array([[4.723,2.53],[4.528,2.644],[4.528,2.868],
#                         [4.723,2.982],[4.921,2.868],[4.921,2.644]])    
#         obstacles = [wo1,wo2,wo3,wo4,wo5,wo6]
        
    if workspace==3: #final project 2d obstacles 
        #xmin,xmax,ymin,ymax
        
        #rectangle "gates"
        wo1 = np.array([4.134,5.315,4.803,4.921])
        wo2 = np.array([4.134,5.315,3.819,3.937])
        wo3 = np.array([1.378,2.559,3.622,3.74])
        wo4 = np.array([1.378,2.559,2.638,2.756])   
        #lower nuts
        wo5 = np.array([1.772,2.165,1.348,1.801])      
        wo6 = np.array([4.528,4.921,2.53,2.982])           
        obstacles = [wo1,wo2,wo3,wo4,wo5,wo6]
        
    if workspace==4:#final project 3d obstacles
        #xmin,xmax,ymin,ymax,zmin,zmax
        
        #rectangle "gates"
        wo1 = np.array([4.134,5.315,4.803,4.921,0,0.866])
        wo2 = np.array([4.134,5.315,3.819,3.937,0,0.866])
        wo3 = np.array([1.378,2.559,3.622,3.74,0,0.866])
        wo4 = np.array([1.378,2.559,2.638,2.756,0,0.866])
        #rectangle "gate tops"
        wo5 = np.array([4.134,5.315,4.803-0.34,4.921,0.866,0.984])
        wo6 = np.array([4.134,5.315,3.819,3.937+0.34,0.866,0.984])
        wo7 = np.array([1.378,2.559,3.622-0.34,3.74,0.866,0.984])
        wo8 = np.array([1.378,2.559,2.638,2.756+0.34,0.866,0.984])       
        #lower nuts
        wo9 = np.array([1.772,2.165,1.348,1.801,0,0.125])      
        wo10 = np.array([4.528,4.921,2.53,2.982,0,0.125])   
        #vert shafts
        wo11 = np.array([1.772+0.108,2.165-0.108,1.348+0.108,1.801-0.108,0,1.174])      
        wo12 = np.array([4.528+0.108,4.921-0.108,2.53+0.108,2.982-0.108,0,1.174])
        #upper nuts
        wo13 = np.array([1.772,2.165,1.348,1.801,1.174,1.174+0.125])      
        wo14 = np.array([4.528,4.921,2.53,2.982,1.174,1.174+0.125]) 
        #upper washers
        wo15 = np.array([1.772-0.276,2.165+0.276,1.348-0.276,1.801+0.276,1.174+0.125,1.174+0.125+0.04])      
        wo16 = np.array([4.528-0.276,4.921+0.276,2.53-0.276,2.982+0.276,1.174+0.125,1.174+0.125+0.04]) 
        #top nuts
        wo17 = np.array([1.772,2.165,1.348,1.801,1.174+0.125+0.04,
                                                 1.174+0.125+0.04+0.125])      
        wo18 = np.array([4.528,4.921,2.53,2.982,1.174+0.125+0.04,
                                                 1.174+0.125+0.04+0.125])        
        
        obstacles = [wo1,wo2,wo3,wo4,wo5,wo6,wo7,wo8,wo9,wo10,wo11,wo12,
                    wo13,wo14,wo15,wo16,wo17,wo18]
  
    if (workspace== 0 or workspace== 1 or 
        workspace== 2 ): 
        triangobs=[]
        for ob in obstacles:
                tri = Delaunay(ob)
                triangobs.append(tri)
                
    if workspace== 3 or  workspace== 4:            
        triangobs=[]         
    return obstacles,triangobs

## radiusofpoints

In [4]:
#creates nested circles of points
def radiusofpoints(x,y,r,resolution=15,radiuslayers=3):
    #x,y are center of circle. r=radius
    xlist=[]
    ylist=[]
    xouterlist=[]
    youterlist=[]    
    for j in np.linspace(0, r, num=radiuslayers):
        for i in range(resolution):
                xnew=x+(j*math.cos(3.14*(i/(resolution/2))))
                ynew=y+(j*math.sin(3.14*(i/(resolution/2))))
                xlist.append(xnew)
                ylist.append(ynew)
                if j==r:
                    xouterlist.append(xnew)
                    youterlist.append(ynew) 
    return xlist,ylist,xouterlist,youterlist

#create nested spheres of points. 
def radiusofpoints3D(x,y,z,r,perimeterresolution,
                         zlayers,inwardlayertotal):
    xlist=[]
    ylist=[]
    zlist=[]
    xlistlower=[] #lower half of sphere
    ylistlower=[]
    zlistlower=[]

    xlistouter=[] #outer shell
    ylistouter=[]
    zlistouter=[]
    xlistouterlower=[]
    ylistouterlower=[]
    zlistouterlower=[]

    for inwardlayer in np.linspace(1,inwardlayertotal,inwardlayertotal):
        rtemp=r*inwardlayer/inwardlayertotal
        for currentradius in np.linspace(0,rtemp,zlayers):
            for i in range(0,perimeterresolution):
                    xnew=x+(currentradius*math.cos(3.14*(i/(perimeterresolution/2))))
                    ynew=y+(currentradius*math.sin(3.14*(i/(perimeterresolution/2))))
                    xnew=xnew
                    ynew=ynew
                    xlist.append(xnew)
                    ylist.append(ynew)
                    xlistlower.append(xnew)
                    ylistlower.append(ynew)
                    znew=(math.sqrt(abs(pow(rtemp, 2)
                                        -pow(xnew-x,2)-pow(ynew-y,2))))
                    zlist.append(z+znew)
                    zlistlower.append((znew*-1)+z)

                    #filter for only outermost layer (for plotting)
                    if inwardlayer==inwardlayertotal:
                        xlistouter.append(xnew)
                        ylistouter.append(ynew)
                        zlistouter.append(z+znew)
                        xlistouterlower.append(xnew)
                        ylistouterlower.append(ynew)   
                        zlistouterlower.append((znew*-1)+z) 

    xlist.extend(xlistlower)
    ylist.extend(ylistlower)   
    zlist.extend(zlistlower)
    xlistouter.extend(xlistouterlower)
    ylistouter.extend(ylistouterlower)   
    zlistouter.extend(zlistouterlower)
    
    return xlist,ylist,zlist,xlistouter,ylistouter,zlistouter

## plotting test

In [5]:
workspace=3
obstacles,triangobs=defineworkspace(workspace)

if workspace== 0 or workspace== 1 or workspace== 2 or workspace== 3: 
    fig, ax = plt.subplots(figsize=(6,6))
    ax.set_aspect('equal', adjustable='box')
    
if workspace==0:
    ax.set_xlim(-1, 11)
    ax.set_ylim(-3, 3)
    start=[0,0]
    goal=[10,0]
if workspace==1:
    ax.set_xlim(-1, 13)
    ax.set_ylim(-1, 13)
    start=[0,0]
    goal=[10,10]
if workspace==2:                          
    ax.set_xlim(-6, 36)
    ax.set_ylim(-6, 6)
    start=[0,0]
    goal=[35,0] 
if workspace==3:                          
    ax.set_xlim(0, 8)
    ax.set_ylim(0, 7)
    start=[6.8385,0.7875]
    goal=[0.63,6.102]
if workspace==4:
    start=[6.8385,0.7875,.25]
    goal=[0.63,6.102,2]      
    
if workspace== 0 or workspace== 1 or workspace== 2: 
    plt.plot(start[0],start[1],'.')
    plt.plot(goal[0],goal[1],'.')   
    plt.text(start[0],start[1],'start')
    plt.text(goal[0],goal[1],'goal')

    for ob in obstacles:
            tri = Delaunay(ob)
            triangobs.append(tri)
            plt.triplot(ob[:,0], ob[:,1], tri.simplices)
            plt.plot(ob[:,0], ob[:,1])
if workspace== 3:
    plt.plot(start[0],start[1],'.')
    plt.plot(goal[0],goal[1],'.')   
    plt.text(start[0],start[1],'start')
    plt.text(goal[0],goal[1],'goal')
    for ob in obstacles:
        #rectangleplot(xmin,xmax,ymin,ymax,Color,Color2,fill=0,Alpha=0)
            rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k',fill=0,Alpha=0)
    plt.show()        
    
if workspace== 4:
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')
    ax.scatter(start[0],start[1],start[2])
    ax.scatter(goal[0],goal[1],goal[2],'.')   
    ax.text(start[0],start[1],start[2],'start')
    ax.text(goal[0],goal[1],goal[2],'goal')
    x=start[0]
    y=start[1]
    z=start[2]
    r=0.07
    #xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,r,resolution=15,radiuslayers=3)
    xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                          x,y,z,r,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
    ax.plot(xlist,ylist,zlist,'.') 
    set_aspect_equal_3d(ax)
    colorcounter=0
    color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
           'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'g','g']
    counter=1
    for ob in obstacles:
            #tri = Delaunay(ob[:,0], ob[:,1])
            
            #print(counter,"   ",ob)
            counter+=1
            rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],color[colorcounter],1,0)
            #ax.plot(ob[:,0], ob[:,1], ob[:,2])    
            colorcounter+=1
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## collisioncheck

In [6]:
#check if the robot at point x1,y1 is in collision
#with any of the obstacles. 

def collisioncheck(x,y,triangobs):
    collision=False
    for t in triangobs:     
        sensor=t.find_simplex([x,y])
        if sensor>-1:
            collision=True
            break
    return collision

#usb cable has cross-sectional diameter of 0.14in



def collisioncheckradius(x,y,r,triangobs):
    xlist,ylist,xouterlist,youterlist=radiusofpoints(x,y,r)
    collision=False 
    for i in range(len(xlist)):
        collision=collisioncheck(xlist[i],ylist[i],triangobs)
        if collision==True:
                break
    return collision

#for new definition of workplace 3
def collisioncheckV2(x,y,obstacles):
    collision=False
    for ob in obstacles:      
        #xmin,xmax,ymin,xmax
        if( (ob[0]<=x  and  x<=ob[1]) and
                (ob[2]<=y and y<=ob[3]))==True:      #if true, then intersection
            collision=True
            #print("collision at ",x,y)
            break
    return collision   

def collisioncheckradiusV2(x,y,radius,obstacles):
    xlist,ylist,xouterlist,youterlist=radiusofpoints(x,y,radius)
    collision=False 
    for i in range(len(xlist)):
        collision=collisioncheckV2(xlist[i],ylist[i],obstacles)
        if collision==True:
                break
    return collision           

def collisioncheck3D(x,y,z,obstacles):
    collision=False
    for ob in obstacles:
        if ((x>=ob[0] and x<=ob[1]) and
           (y>=ob[2] and y<=ob[3]) and
            (z>=ob[4] and z<=ob[5])):
            collision=True
            #print("collision with obstacle at",ob)
            break
    return collision

def collisioncheckradius3D(x,y,z,radius,obstacles):
    #check for collisions in sphere around central point in the arguments
    xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(x,y,z,radius,
                        perimeterresolution=5,zlayers=8,inwardlayertotal=1)
    collision=False
    for i in range(len(xlist)):
        collision=collisioncheck3D(x,y,z,obstacles)
        if collision==True:
            #print("collision with obstacle at",ob)
            break
    return collision
 
def prioritysize(inputlist):
    return inputlist[3]
def neighbordist(inputlist):
    return inputlist[1]


## generate node

In [7]:
def generate_node(workspace):    
    
    seed(time.time())
    if workspace==0:
        x_range=12
        y_range=6
        x_adjust=1
        y_adjust=3
    if workspace==1:
        x_range=14
        y_range=14
        x_adjust=1
        y_adjust=1
    if workspace==2:
        x_range=42
        y_range=12
        x_adjust=6
        y_adjust=6                             
    if workspace==3:
        x_range=8
        y_range=7
        x_adjust=0
        y_adjust=0
    xval=np.round((random()*x_range),1)
    yval=np.round((random()*y_range),1)
    xval=xval-x_adjust  
    yval=yval-y_adjust
           
    return xval,yval

#Used for syclop
def generate_nodezone(xmin,xmax,ymin,ymax):    
    #this function presumes that min and max values are all positive.
    
    #I debated using normal distributions, and may eventually implement. 
    #x = np.random.normal(0.5,1)      # mean 0.5 and cov 1
    
    seed(time.time())
    x_range=(xmax-xmin)
    y_range=(ymax-ymin)
    xval=np.round((random()*x_range),1)
    yval=np.round((random()*y_range),1)
    xval=xval+xmin  
    yval=yval+ymin
    
    return xval,yval
def generate_node3D():      
    seed(time.time())                      
    if workspace==4:
        x_range=8
        y_range=7
        z_range=4
    xval=np.round((random()*x_range),1)
    yval=np.round((random()*y_range),1)
    zval=np.round((random()*z_range),1)
 
    return xval,yval,zval

In [8]:
def nearestpoint(xstart,ystart,xgoal,ygoal,r):
    
    totaldist=math.sqrt(((xstart-xgoal)**2)+
                               ((ystart-ygoal)**2))
    if totaldist==0:
        xgoal,ygoal=generate_node(workspace)
        totaldist=math.sqrt(((xstart-xgoal)**2)+
                               ((ystart-ygoal)**2))
        
    t=r/totaldist
    
    xnear=((1-t)*xstart )+(t*xgoal)
    ynear=((1-t)*ystart )+(t*ygoal)
    
    return xnear,ynear

def nearestpoint3D(xstart,ystart,zstart,xgoal,ygoal,zgoal,r):
    
    totaldist=math.sqrt(((xstart-xgoal)**2)+
                               ((ystart-ygoal)**2)+((zstart-zgoal)**2))
    if totaldist==0:
        xgoal,ygoal,zgoal =generate_node3D()
        totaldist=math.sqrt(((xstart-xgoal)**2)+
                               ((ystart-ygoal)**2)+((zstart-zgoal)**2))
    #linear interpolation eqtn    
    t=r/totaldist
    
    xnear=((1-t)*xstart )+(t*xgoal)
    ynear=((1-t)*ystart )+(t*ygoal)
    znear=((1-t)*zstart )+(t*zgoal)
    return xnear,ynear,znear

## IsSubpathCollisionFree

In [9]:
def IsSubpathCollisionFree(x1,y1,x2,y2,obstacles,radius):
        pathstatus=True
        #r=0.07
        if (x2-x1)!=0:
            a=(y2-y1)/(x2-x1)
            b=y2-(a*x2)
            #y=ax+b
            for xtemp in np.linspace(x1, x2,num=8):
                ytemp=(a*xtemp)+b
                if collisioncheckradius(xtemp,ytemp,radius,triangobs)==True:
                #if collisioncheck(xtemp,ytemp,triangobs)==True:
                       pathstatus=False
        else:
             for ytemp in np.linspace(y1, y2,num=8):
                #if collisioncheck(x1,ytemp,triangobs)==True:
                if collisioncheckradius(x1,ytemp,radius,triangobs)==True:
                       pathstatus=False     
        return pathstatus #T/F
    
def IsSubpathCollisionFreeV2(x1,y1,x2,y2,obstacles,radius):
        pathstatus=True
        #r=0.07
        if (x2-x1)!=0:
            a=(y2-y1)/(x2-x1)
            b=y2-(a*x2)
            #y=ax+b
            for xtemp in np.linspace(x1, x2,num=8):
                ytemp=(a*xtemp)+b
                if collisioncheckradiusV2(xtemp,ytemp,radius,obstacles)==True:
                #if collisioncheck(xtemp,ytemp,triangobs)==True:
                       pathstatus=False
        else:
             for ytemp in np.linspace(y1, y2,num=8):
                #if collisioncheck(x1,ytemp,triangobs)==True:
                if collisioncheckradiusV2(x1,ytemp,radius,obstacles)==True:
                       pathstatus=False     
        return pathstatus #T/F    
    
def IsSubpathCollisionFree3D(x1,y1,z1,x2,y2,z2,obstacles,radius):
        pathstatus=True
        #print("testing subpaths from",x1,y1,z1,"to",x2,y2,z2)
        for percent in range(1,100):
            #print (percent)
            if (x2-x1)!=0 :
                xtemp=x1+((percent/100)*(x2-x1))
            else:
                xtemp=x1  
            if (y2-y1)!=0 :
                ytemp=y1+((percent/100)*(y2-y1))
            else:
                ytemp=y1     
            if (z2-z1)!=0 :
                ztemp=z1+((percent/100)*(z2-z1))
            else:
                ztemp=z1  
            #print("colcheck at ",xtemp,ytemp,ztemp )   
            
            #if collisioncheck3D(xtemp,ytemp,ztemp,obstacles)==True:
            if collisioncheckradius3D(xtemp,ytemp,ztemp,radius,obstacles)==True:    
                
                pathstatus=False
                #print("collision at",xtemp,ytemp,ztemp)
                break
                        
        return pathstatus #T/F

## goalbias rrt

In [10]:
# n =5000, r = 0.5, p goal = 0.05, and epsilon= 0.25.
#r= step distance
#p_goal= goal bias probability
#epsilon=acceptable distance to goal to end algorithm
#n=max iterations
#radius= radius to check  as path grows (creates volume)

def goalbiasRRT(workspace,start,goal,r,p_goal,epsilon,radius,n,plotting):
    start_time = time.time()
    #print("goal:",goal)
    obstacles,triangobs=defineworkspace(workspace)

    xylist=[]
    Treelist=[]  #indicies in xylist  [parentnode,childnode]

    xylist.append([start[0],start[1]])

    counter=0
    while True:
        goalbiasval=random()
        #print(goalbiasval)
        if goalbiasval<p_goal:
            #print("walking to goal")
            xnode=goal[0]
            ynode=goal[1]
        else:
            xnode,ynode=generate_node(workspace)  

        mindistindex=0
        mindist=99999999999
        for i in range(len(xylist)):
            totaldist=math.sqrt(((xylist[i][0]-xnode)**2)+
                                   ((xylist[i][1]-ynode)**2))  
            if totaldist<mindist:
                    mindistindex=i
                    mindist=totaldist

        xnear,ynear=nearestpoint(xylist[mindistindex][0],
                        xylist[mindistindex][1],xnode,ynode,r)

        if workspace ==0 or workspace ==1 or workspace ==2 :
            colstatus=IsSubpathCollisionFree(xylist[mindistindex][0],
                               xylist[mindistindex][1],xnear,ynear,obstacles,radius)
        if workspace ==3:
            colstatus=IsSubpathCollisionFreeV2(xylist[mindistindex][0],
                               xylist[mindistindex][1],xnear,ynear,obstacles,radius)    
            
        if colstatus==True:
            xylist.append([xnear,ynear])
            Treelist.append([mindistindex,len(xylist)-1])

        goaldist=math.sqrt(((xnear-goal[0])**2)+
                                   ((ynear-goal[1])**2))
        if goaldist<epsilon:
            #print("reached goal!")
            break
        if counter>n:
            print("counter hit maximum of",n)
            return 0,0,0
            break

        counter+=1

    #print("stopped at counter",counter)

    #parent,child

    #############Generate path####################
    pathlist=[]
    pathlist.append(Treelist[(len(Treelist)-1)])

    while True:
        for i in range(len(Treelist)):
            if Treelist[i][1]==pathlist[len(pathlist)-1][0]:
                    pathlist.append(Treelist[i])
                    break
        if pathlist[len(pathlist)-1][0]==0:
            break


        totaldist=0
        for g in range(len(pathlist)-1):
                index1=pathlist[g][0]
                index2=pathlist[g][1]
                x1=xylist[index1][0]
                y1=xylist[index1][1]
                x2=xylist[index2][0]
                y2=xylist[index2][1]
                totaldist+=math.sqrt(((x1-x2)**2)+
                                       ((y1-y2)**2))

    if plotting ==1:
        fig, ax = plt.subplots(figsize=(6,6))
        if workspace==0:
            ax.set_xlim(-1, 11)
            ax.set_ylim(-3, 3)
        if workspace==1:
            ax.set_xlim(-1, 13)
            ax.set_ylim(-1, 13)
        if workspace==2:                          
            ax.set_xlim(-6, 36)
            ax.set_ylim(-6, 6)
        for g in range(len(Treelist)):
            index1=Treelist[g][0]
            index2=Treelist[g][1]
            x1=xylist[index1][0]
            y1=xylist[index1][1]
            x2=xylist[index2][0]
            y2=xylist[index2][1]
            plt.plot([x1,x2],[y1,y2],linewidth=1)
        for g in range(len(pathlist)):
            index1=pathlist[g][0]
            index2=pathlist[g][1]
            x1=xylist[index1][0]
            y1=xylist[index1][1]
            x2=xylist[index2][0]
            y2=xylist[index2][1]
            plt.plot([x1,x2],[y1,y2],linewidth=4,color='r')
        for ob in obstacles:
                tri = Delaunay(ob)
                triangobs.append(tri)
                plt.triplot(ob[:,0], ob[:,1], tri.simplices,color='k')
                plt.plot(ob[:,0], ob[:,1],color='k')
        #plt.title("GoalBiasRRT  PathLength: "+str(totaldist))
        ax.set_aspect('equal', adjustable='box')
    
    outputpathlist=[]
    
    for g in range(len(pathlist)-1,-1,-1):
            index1=pathlist[g][0]
            index2=pathlist[g][1]
            x1=xylist[index1][0]
            y1=xylist[index1][1]
            #x2=xylist[index2][0]
            #y2=xylist[index2][1]
            outputpathlist.append([x1,y1])
            if g==0:
                x2=xylist[index2][0]
                y2=xylist[index2][1]
                outputpathlist.append([x2,y2])
            
    #elapsed_time = time.time() - start_time    
    
    return outputpathlist

## goalbiasRRT3D

In [11]:
# n =5000, r = 0.5, p goal = 0.05, and epsilon= 0.25.
#n=max iterations
#r= step distance
#p_goal= goal bias probability
#epsilon=acceptable distance to goal to end algorithm

def goalbiasRRT3D(workspace,start,goal,r,p_goal,epsilon,radius,n,plotting):
    start_time = time.time()
    #print("goal:",goal)
    obstacles,triangobs=defineworkspace(workspace)

    xyzlist=[]
    Treelist=[]  #indicies in xylist  [parentnode,childnode]

    xyzlist.append([start[0],start[1],start[2]])

    counter=0
    while True:
        goalbiasval=random()
        #print(goalbiasval)
        if goalbiasval<p_goal:
            #print("walking to goal")
            xnode=goal[0]
            ynode=goal[1]
            znode=goal[2]
        else:
            xnode,ynode,znode=generate_node3D()  

        mindistindex=0
        mindist=99999999999
        for i in range(len(xyzlist)):
            totaldist=math.sqrt(((xyzlist[i][0]-xnode)**2)+
                                   ((xyzlist[i][1]-ynode)**2)+
                                    ((xyzlist[i][2]-znode)**2))  
            if totaldist<mindist:
                    mindistindex=i
                    mindist=totaldist

        xnear,ynear,znear=nearestpoint3D(xyzlist[mindistindex][0],
                        xyzlist[mindistindex][1],xyzlist[mindistindex][2],
                                         xnode,ynode,znode,r)
        
        colstatus=IsSubpathCollisionFree3D(xyzlist[mindistindex][0],
                               xyzlist[mindistindex][1],xyzlist[mindistindex][2],
                                         xnear,ynear,znear,obstacles,radius)
        
        if colstatus==True:
            xyzlist.append([xnear,ynear,znear])
            Treelist.append([mindistindex,len(xyzlist)-1])

        goaldist=math.sqrt(((xnear-goal[0])**2)+
                           ((ynear-goal[1])**2)+
                           ((znear-goal[2])**2))
        if goaldist<epsilon:
            #print("reached goal!")
            break
        if counter>n:
            print("counter hit maximum of",n)
            return 0,0,0
            break

        counter+=1

    #print("stopped at counter",counter)

    #parent,child


    #############Generate path####################
    pathlist=[]
    pathlist.append(Treelist[(len(Treelist)-1)])

    while True:
        for i in range(len(Treelist)):
            if Treelist[i][1]==pathlist[len(pathlist)-1][0]:
                    pathlist.append(Treelist[i])
                    break
        if pathlist[len(pathlist)-1][0]==0:
            break
            
    totaldist=0
    for g in range(len(pathlist)-1):
                index1=pathlist[g][0]
                index2=pathlist[g][1]
                x1=xyzlist[index1][0]
                y1=xyzlist[index1][1]
                z1=xyzlist[index1][1]
                x2=xyzlist[index2][0]
                y2=xyzlist[index2][1]
                z2=xyzlist[index2][2]
                totaldist+=math.sqrt(((x1-x2)**2)+
                                       ((y1-y2)**2)+
                                     ((z1-z2)**2))
   
    #elapsed_time = time.time() - start_time    
    
#     if plotting ==1:
#         print("local plotting actions")
      
    
    
    outputpathlist=[]
    for g in range(len(pathlist)-1,-1,-1):
                index1=pathlist[g][0]
                index2=pathlist[g][1]
                x1=xyzlist[index1][0]
                y1=xyzlist[index1][1]
                z1=xyzlist[index1][2]
                outputpathlist.append([x1,y1,z1])
                if g==0:
                    x2=xyzlist[index2][0]
                    y2=xyzlist[index2][1]
                    z2=xyzlist[index2][2]
                    outputpathlist.append([x2,y2,z2])        
            
    return outputpathlist

## rrt with no waypoints 

In [12]:
# now = datetime.now()
# current_time = now.strftime("%H:%M:%S")
# print("Start Time =", current_time)

#n=max iterations
#r= step distance
#p_goal= goal bias probability
#epsilon=acceptable distance to goal to end algorithm
#radius=object (wire) radius

r = 0.25
p_goal = 0.15
epsilon= 0.1
n =5000
radius=0.07
workspace=3

if workspace==0: 
    start=[0,0]
    goal=[10,0] 
if workspace==1:
    start=[0,0]
    goal=[10,10]
if workspace==2:
    start=[0,0]
    goal=[35,0]   
if workspace==3:
    start=[6.8385,0.7875]
    goal=[0.63,6.102]  
    
    #first two waypoints:
    #start=[6.8385,0.7875]
    #goal=[6,3.3]
if workspace==4:
    start=[6.8385,0.7875,0.25]
    #goal=[0.63,6.102,0.2] 
    goal=[4.75,4.25,0.2]

obstacles,triangobs=defineworkspace(workspace)
if workspace ==0 or workspace ==1 or workspace ==2:
    outputpathlist=goalbiasRRT(
                    workspace,start,goal,r,p_goal,epsilon,radius,n,0)
    #print("outputpathlist",outputpathlist)
    fig, ax = plt.subplots(figsize=(6,6))
    
    for n in range(len(outputpathlist)-1): 
            x1path=outputpathlist[n][0]
            y1path=outputpathlist[n][1]
            x2path=outputpathlist[n+1][0]
            y2path=outputpathlist[n+1][1]  
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],linewidth=4,color='r', 
                    zorder=-5) 
            x=x1path
            y=y1path
            xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,radius,resolution=15,radiuslayers=3)
            plt.plot(xlist,ylist,'.')   
    for ob in obstacles:
        tri = Delaunay(ob)
        triangobs.append(tri)
        plt.triplot(ob[:,0], ob[:,1], tri.simplices,color='k')
        plt.plot(ob[:,0], ob[:,1],color='k')
        #plt.title("GoalBiasRRT  PathLength: "+str(totaldist))
    
    ax.set_aspect('equal', adjustable='box')
if workspace ==3:   
    outputpathlist=goalbiasRRT(
                    workspace,start,goal,r,p_goal,epsilon,radius,n,0)
    #print("outputpathlist",outputpathlist)
    fig, ax = plt.subplots(figsize=(6,6))
    
    for n in range(len(outputpathlist)-1): 
            x1path=outputpathlist[n][0]
            y1path=outputpathlist[n][1]
            x2path=outputpathlist[n+1][0]
            y2path=outputpathlist[n+1][1]  
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],linewidth=4,color='r', 
                    zorder=-5) 
            x=x1path
            y=y1path
            xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,radius,resolution=15,radiuslayers=3)
            plt.plot(xlist,ylist,'.')  
    x=x2path
    y=y2path        
    xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,radius,resolution=15,radiuslayers=3)
    plt.plot(xlist,ylist,'.')
    for ob in obstacles:
        rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k',1,0)    
    plt.show()
if workspace ==4:
    outputpathlist=goalbiasRRT3D(
                    workspace,start,goal,r,p_goal,epsilon,radius,n,1)  
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')
    ax.scatter(start[0],start[1],start[2])
    ax.scatter(goal[0],goal[1],goal[2],'.')   
    ax.text(start[0],start[1],start[2],'start')
    ax.text(goal[0],goal[1],goal[2],'goal')
    set_aspect_equal_3d(ax)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    
    colorcounter=0
    color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
           'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'b','b']
    for ob in obstacles:
            rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],color[colorcounter],1,0)
            colorcounter+=1
            
    #ax.plot(outputpathlist[:,0], outputpathlist[:,1], outputpathlist[:,2]) 
    for n in range(len(outputpathlist)-1): 
            x1path=outputpathlist[n][0]
            y1path=outputpathlist[n][1]
            z1path=outputpathlist[n][2]
            x2path=outputpathlist[n+1][0]
            y2path=outputpathlist[n+1][1]
            z2path=outputpathlist[n+1][2]
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='b', 
                    zorder=-5) 
            x=x1path
            y=y1path
            z=z1path
            xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                          x,y,z,r,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
            ax.plot(xlist,ylist,zlist,'.') 
            
            if n == len(outputpathlist)-1:
                x=x2path
                y=y2path
                z=z2path
                xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                              x,y,z,r,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
                ax.plot(xlist,ylist,zlist,'.') 

  
    plt.show()     
#     now = datetime.now()
#     current_time = now.strftime("%H:%M:%S")
#     print("Completion Time =", current_time)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## rrt with waypoints 

In [13]:
r = 0.25
p_goal = 0.05
epsilon= 0.25
n =5000
workspace=3
radius=0.07

obstacles,triangobs=defineworkspace(workspace)

if workspace==3:
    start=[6.8385,0.7875]
    goal=[0.63,6.102]  

#larger list
    targetlist=[[6.8385,0.7875],[6,3.3],[4,3.3],[3,1],[1.5,1],
        [1.5,2.25],[2.75,2.25],[3.75,4.25],[5.75,4.25],[6.25,5.5],
        [3.75,5.5],[2.5,3.25],[1,3.25],[0.63,6.102]]  

#smaller waypoint list    
#     targetlist=[[6.8385,0.7875],[4.75,3.25],[4.25,2.75],[2.75,0.75],[1.25,1.25],
#         [2.75,2.25],[4.75,4.25],[5.25,5.25],[1.25,3.25],[0.63,6.102]] 
    
if workspace==4:
    start=[6.8385,0.7875,0.25]
    #goal=[0.63,6.102,0.2] 
    goal=[0.63,6.102,0.25]
    targetlist=[[6.8385,0.7875,0.5],[6,3.3,0.5],[4,3.3,0.5],[3,1,0.5],[1.5,1,0.5],
        [1.5,2.25,0.5],[2.75,2.25,0.5],[3.75,4.5,0.5],[5.75,4.25,0.5],[6.25,5.5,0.5],
        [3.75,5.5,0.5],[2.5,3.25,0.5],[1,3.25,0.5],[0.63,6.102,0.5]]  
mainpath=[]

if workspace ==0 or workspace ==1 or workspace ==2 or workspace ==3:
    for g in range(len(targetlist)-1):
        outputpathlist=goalbiasRRT(
                        workspace,targetlist[g],targetlist[g+1],r,p_goal,epsilon,radius,n,0)
        mainpath.extend(outputpathlist)   
        
    fig, ax = plt.subplots(figsize=(6,6))
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                plt.plot([x1,x2],[y1,y2],linewidth=4,color='g')
    for ob in obstacles:
                rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k',1,0)    
    for n in range(len(mainpath)-1): 
                x1path=mainpath[n][0]
                y1path=mainpath[n][1]
                x2path=mainpath[n+1][0]
                y2path=mainpath[n+1][1]
                #print("line from ",x1path,y1path," to ",x2path,y2path)
                plt.plot([x1path,x2path],[y1path,y2path],linewidth=4,color='r')   
                x=x1path
                y=y1path
                xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,0.07,resolution=15,radiuslayers=3)
                plt.plot(xlist,ylist,'.')

    plt.title("GoalBias RRT with Waypoints and Volume Collision Checking")
    ax.set_aspect('equal', adjustable='box')

    
if workspace ==4:
    for g in range(len(targetlist)-1):
        outputpathlist=goalbiasRRT3D(
                        workspace,targetlist[g],targetlist[g+1],r,p_goal,epsilon,radius,n,0)
        #print(outputpathlist)
        mainpath.extend(outputpathlist) 
        
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')
    ax.scatter(start[0],start[1],start[2])
    ax.scatter(goal[0],goal[1],goal[2],'.')   
    ax.text(start[0],start[1],start[2],'start')
    ax.text(goal[0],goal[1],goal[2],'goal')
    set_aspect_equal_3d(ax)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    
    
    colorcounter=0
    color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
           'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'b','b']
    for ob in obstacles:
            rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],
                            color[colorcounter],1,0)
            colorcounter+=1
            
    #ax.plot(outputpathlist[:,0], outputpathlist[:,1], outputpathlist[:,2]) 
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                z1=targetlist[g][2]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                z2=targetlist[g+1][2]
                plt.plot([x1,x2],[y1,y2],[z1,z2],linewidth=4,color='g')
                
    for n in range(len(mainpath)-1): 
            x1path=mainpath[n][0]
            y1path=mainpath[n][1]
            z1path=mainpath[n][2]
            x2path=mainpath[n+1][0]
            y2path=mainpath[n+1][1]
            z2path=mainpath[n+1][2]
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='b', 
                    zorder=-5)  
            x=x1path
            y=y1path
            z=z1path
            xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                          x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
            ax.plot(xlist,ylist,zlist,'.') 
            
            if n == len(outputpathlist)-1:
                x=x2path
                y=y2path
                z=z2path
                xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                              x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
                ax.plot(xlist,ylist,zlist,'.') 
       
    plt.show()   


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## rrt with waypoints 3D

In [16]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

r = 0.25
p_goal = 0.1
epsilon= 0.25
n =5000
workspace=4
radius=0.07

obstacles,triangobs=defineworkspace(workspace)
#print("test")
if workspace==3:
    start=[6.8385,0.7875]
    goal=[0.63,6.102]  
    
    targetlist=[[6.8385,0.7875],[6,3.3],[4,3.3],[3,1],[1.5,1],
        [1.5,2.25],[2.75,2.25],[3.75,4.25],[5.75,4.25],[6.25,5.5],
        [3.75,5.5],[2.5,3.25],[1,3.25],[0.63,6.102]]  
    
if workspace==4:
    start=[6.8385,0.7875,0.25]
    #goal=[0.63,6.102,0.2] 
    goal=[0.63,6.102,0.25]
    targetlist=[[6.8385,0.7875,0.5],[6,3.3,0.5],[4,3.3,0.5],[3,1,0.5],[1.5,1,0.5],
        [1.5,2.25,0.5],[2.75,2.25,0.5],[3.75,4.25,0.5],[5.75,4.25,0.5],[6.25,5.5,0.5],
        [3.75,5.5,0.5],[2.5,3.25,0.5],[1,3.25,0.5],[0.63,6.102,0.5]]  
    

mainpath=[]

if workspace ==0 or workspace ==1 or workspace ==2 or workspace ==3:
    for g in range(len(targetlist)-1):
        outputpathlist=goalbiasRRT(
                        workspace,targetlist[g],targetlist[g+1],r,p_goal,epsilon,radius,n,0)
        #print(outputpathlist)
        mainpath.extend(outputpathlist)   
        
    fig, ax = plt.subplots(figsize=(6,6))
    for g in range(len(targetlist)-1):
                print("segment ", g)
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                plt.plot([x1,x2],[y1,y2],linewidth=4,color='g')
    for ob in obstacles:
                    tri = Delaunay(ob)
                    triangobs.append(tri)
                    plt.triplot(ob[:,0], ob[:,1], tri.simplices,color='k')
                    plt.plot(ob[:,0], ob[:,1],color='k')
    for n in range(len(mainpath)-1): 
                x1path=mainpath[n][0]
                y1path=mainpath[n][1]
                x2path=mainpath[n+1][0]
                y2path=mainpath[n+1][1]
                #print("line from ",x1path,y1path," to ",x2path,y2path)
                plt.plot([x1path,x2path],[y1path,y2path],linewidth=4,color='r')   
                x=x1path
                y=y1path
                xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,radius,resolution=15,radiuslayers=3)
                plt.plot(xlist,ylist,'.')

    plt.title("Intended path with waypoints")
    ax.set_aspect('equal', adjustable='box')

    
if workspace ==4:
    for g in range(len(targetlist)-1):
        outputpathlist=goalbiasRRT3D(
                        workspace,targetlist[g],targetlist[g+1],r,p_goal,epsilon,radius,n,1)
        #print(outputpathlist)
        mainpath.extend(outputpathlist) 
        
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')
    ax.scatter(start[0],start[1],start[2])
    ax.scatter(goal[0],goal[1],goal[2],'.')   
    ax.text(start[0],start[1],start[2],'start')
    ax.text(goal[0],goal[1],goal[2],'goal')
    set_aspect_equal_3d(ax)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    
    
    colorcounter=0
    color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
           'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'b','b']
    for ob in obstacles:
            rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],
                            color[colorcounter],1,0)
            colorcounter+=1
            
    #ax.plot(outputpathlist[:,0], outputpathlist[:,1], outputpathlist[:,2]) 
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                z1=targetlist[g][2]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                z2=targetlist[g+1][2]
                plt.plot([x1,x2],[y1,y2],[z1,z2],linewidth=4,color='g')
                
    for n in range(len(mainpath)-1): 
            x1path=mainpath[n][0]
            y1path=mainpath[n][1]
            z1path=mainpath[n][2]
            x2path=mainpath[n+1][0]
            y2path=mainpath[n+1][1]
            z2path=mainpath[n+1][2]
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='b', 
                    zorder=-5)  
            x=x1path
            y=y1path
            z=z1path
            xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                          x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
            ax.plot(xlist,ylist,zlist,'.') 
            
            if n == len(outputpathlist)-1:
                x=x2path
                y=y2path
                z=z2path
                xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                              x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
                ax.plot(xlist,ylist,zlist,'.') 
       
    plt.show()  
    
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 23:50:43


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Current Time = 23:51:12


In [17]:


fig = plt.figure(figsize=(8,8))
ax = fig.gca(projection='3d')
ax.scatter(start[0],start[1],start[2])
ax.scatter(goal[0],goal[1],goal[2],'.')   
ax.text(start[0],start[1],start[2],'start')
ax.text(goal[0],goal[1],goal[2],'goal')
set_aspect_equal_3d(ax)
ax.set_xlabel('$X$')
ax.set_ylabel('$Y$')
ax.set_zlabel('$Z$')


colorcounter=0
color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
       'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'b','b']
for ob in obstacles:
        rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],
                        color[colorcounter],1,0)
        colorcounter+=1

#ax.plot(outputpathlist[:,0], outputpathlist[:,1], outputpathlist[:,2]) 
for g in range(len(targetlist)-1):
            x1=targetlist[g][0]
            y1=targetlist[g][1]
            z1=targetlist[g][2]
            x2=targetlist[g+1][0]
            y2=targetlist[g+1][1]
            z2=targetlist[g+1][2]
            plt.plot([x1,x2],[y1,y2],[z1,z2],linewidth=4,color='g')
            #plt.scatter([x1,x2],[y1,y2],[z1,z2],linewidth=4,color='r')
for n in range(len(mainpath)-1): 
        x1path=mainpath[n][0]
        y1path=mainpath[n][1]
        z1path=mainpath[n][2]
        x2path=mainpath[n+1][0]
        y2path=mainpath[n+1][1]
        z2path=mainpath[n+1][2]
        #print("line from ",x1path,y1path," to ",x2path,y2path)
        if n%2==0:
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='b', 
                zorder=-5)  
        else:
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='y', 
                zorder=-5)       
            
#         x=x1path
#         y=y1path
#         z=z1path
#         xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
#                       x,y,z,0.07,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
#         ax.plot(xlist,ylist,zlist,'.') 

#         if n == len(outputpathlist)-1:
#             x=x2path
#             y=y2path
#             z=z2path
#             xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
#                           x,y,z,0.07,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
#             ax.plot(xlist,ylist,zlist,'.') 

plt.show()   

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## ---Syclop ----

## generate grid

In [18]:
def generategrid(workspace,gridsize):  #generate syclop grid
    if  workspace==3:
        xgridlist=np.arange(0,8+gridsize,gridsize)
        ygridlist=np.arange(0,8+gridsize,gridsize)
        matrix=[]
        for x in range(len(xgridlist)-1):
            for y in range(len(ygridlist)-1):
                    #xmin,xmax,ymin,ymax,dist priority(dist to home) 
                    #manhattan dist between any two zones is gridsize*2
                    matrix.append([xgridlist[x],xgridlist[x+1],
                                                ygridlist[y],ygridlist[y+1]])
        return matrix    

#     if  workspace==4:
#         xgridlist=np.arange(0,8+gridsize,gridsize)
#         ygridlist=np.arange(0,8+gridsize,gridsize)
#         zgridlist=np.arange(0,3+gridsize,gridsize)
#         #print(zgridlist)

#         #xmin,xmax,ymin,ymax,zmin,zmax

#         syclop_highlvl_matrix=[]
#         for x in range(len(xgridlist)-1):
#             for y in range(len(ygridlist)-1):
#                 for z in range(len(zgridlist)-1):
#                     syclop_highlvl_matrix.append([xgridlist[x],xgridlist[x+1],
#                                                 ygridlist[y],ygridlist[y+1],
#                                                  zgridlist[z],zgridlist[z+1]])
        #print(len(syclop_highlvl_matrix))      
        return syclop_highlvl_matrix
    
def gridfilter(workspace,obstacles,gridsize,sgrid):
    #print("grid cells",len(sgrid))
    #print("obstacles",len(obstacles))
    celladd=0
    sgrid2=[]
    sgrid3=[]
    for cell in sgrid: 
        celladd=0
        for ob in obstacles:    
            #if no intersection, cell is valid. Add it to sgrid2           
            if( (((ob[0]<=cell[0]  and  cell[0]<=ob[1]) or
                         (ob[0]<=cell[1] and cell[1]<=ob[1])) and
                ((ob[2]<=cell[2]  and  cell[2]<=ob[3]) or
                         (ob[2]<=cell[3] and cell[3]<=ob[3]))) or
                (((cell[0]<=ob[0]  and  ob[0]<=cell[1]) or
                         (cell[0]<=ob[1] and ob[1]<=cell[1])) and
                ((cell[2]<=ob[2]  and  ob[2]<=cell[3]) or
                         (cell[2]<=ob[3] and ob[3]<=cell[3]))) or
               (((ob[0]<=cell[0]  and  cell[0]<=ob[1]) or
                         (ob[0]<=cell[1] and cell[1]<=ob[1])) and
                ((cell[2]<=ob[2]  and  ob[2]<=cell[3]) or
                         (cell[2]<=ob[3] and ob[3]<=cell[3]))) or 
               (((cell[0]<=ob[0]  and  ob[0]<=cell[1]) or
                         (cell[0]<=ob[1] and ob[1]<=cell[1])) and
               ((ob[2]<=cell[2]  and  cell[2]<=ob[3]) or
                         (ob[2]<=cell[3] and cell[3]<=ob[3]))))==True:      #if true, then intersection
                celladd=1
                break
        if celladd==0:    
            sgrid3.append(cell)
        if celladd==1:
            sgrid2.append(cell)
    #print("sgrid2",len(sgrid2))
    #print("sgrid3",len(sgrid3))
    return sgrid3

## findneighbors

In [19]:
def findneighbors(workspace,gridsize,matrix,startindex,mode):
    neighborlist=[]
    if workspace==3:
        for i in range(len(matrix)):
            #Xmin,Xmax,Ymin,Ymax
            
            #diagonals mode was having issues with part of the map due to an unknown bug. 
            if mode==1 or mode==2:
                if(((matrix[startindex][0] == matrix[i][0] and matrix[startindex][1] == matrix[i][1] )  and #line up vertically
                     (matrix[startindex][2] == matrix[i][3] or matrix[startindex][3] == matrix[i][2] )) # neighbor ymin= startindex ymax or vice versa
                    or
                    ((matrix[startindex][2] == matrix[i][2] and matrix[startindex][3] == matrix[i][3] ) and #line up horizontally
                    (matrix[startindex][0] == matrix[i][1] or matrix[startindex][1] == matrix[i][0] ))): # neighbor xmin= startindex xmax or vice versa
                    neighborlist.append([i,gridsize]) #manhattan dist between cell centers= gridsize
            
            #diagonals
            if mode==2:
                if  ((matrix[startindex][0] == matrix[i][1] and matrix[startindex][2] == matrix[i][3]) or
                    (matrix[startindex][0] == matrix[i][1] and matrix[startindex][3] == matrix[i][2]) or
                     (matrix[startindex][1] == matrix[i][0] and matrix[startindex][2] == matrix[i][3]) or
                     (matrix[startindex][1] == matrix[i][0] and matrix[startindex][3] == matrix[i][2])):

                     # neighbor xmin= startindex xmax or vice versa
                    neighborlist.append([i,((math.sqrt(2)*gridsize))])  #diagonal distance: sqrt(2)*gridsize

    return neighborlist

def coordinatetomatrixindex(matrix,x,y):
    errorcheck=0
    for i in range(len(matrix)):
        if (matrix[i][0]<x and x<matrix[i][1] and
           matrix[i][2]<y and y<matrix[i][3]):
            errorcheck=1
            return i
            break
    
    if errorcheck==0:
        print("error-unable to find valid zone")
        return 0
def coordinatetomatrixindex3D(matrix,x,y,z):
    errorcheck=0
    for i in range(len(matrix)):
        if (matrix[i][0]<x and x<matrix[i][1] and
           matrix[i][2]<y and y<matrix[i][3] and
           matrix[i][4]<z and z<matrix[i][5]):
            errorcheck=1
            return i
            break
    if errorcheck==0:
        print("error-unable to find valid zone")
        
def edgelistgenerate(workspace,gridsize,matrix,neighbormode):
    #startindex=coordinatetomatrixindex(matrix,xstart,ystart)   
    #goalindex=coordinatetomatrixindex(matrix,xgoal,ygoal)  
    edgelist=[] #Initial edge list
   
    
    #Create initial nodes list
    for i in range(len(matrix)):
         neighborlist=findneighbors(workspace,gridsize,matrix,i,neighbormode)
         for j in range(len(neighborlist)): 
                edgelist.append([i,neighborlist[j][0],neighborlist[j][1],0]) #4th value is priority, to be calc'd later
                #print(neighborlist[j][0],neighborlist[j])
    return edgelist

In [20]:
def prioritysize(inputlist):
    return inputlist[3]

def lengthtostart(edgelist,matrix,gridsize,O_list,C_list,startnode,goalnode,Astarmode=0):    
    #O_list=[] #priorityqueue
    #C_list=[] #processednodes
    V=matrix
    pathlist=[]
    totaldist=0
    #print("startnode",startnode,"goalnode",goalnode)
    #O_list.sort(key=prioritysize)
    #C_list.sort(key=prioritysize)
    #add the goal and 2nd to last node to the pathlist
    for j in range(len(O_list)): 
        if (O_list[j][0]== goalnode or O_list[j][1]== goalnode):
        #if (O_list[j][1]== goalnode):    
            pathlist.append(O_list[j][1])
            pathlist.append(O_list[j][0])
            break 
    while True:
        for i in range(len(C_list)):
            #walk back the path to the start
            if C_list[i][1]==pathlist[len(pathlist)-1]:  #if an edge in C list contains 2nd to last node to the pathlist, add this edge to pathlist
                pathlist.append(C_list[i][0])
                #totaldist+=C_list[i][2]
                break
        if pathlist[len(pathlist)-1]==startnode:
            #print("returned to start")
            break
    
    for g in range(len(pathlist)-1):
            index1=pathlist[g]
            index2=pathlist[g+1]

            x1=(V[index1][0] +V[index1][1])/2
            y1=(V[index1][2] +V[index1][3])/2
            x2=(V[index2][0] +V[index2][1])/2
            y2=(V[index2][2] +V[index2][3])/2
            totaldist+=math.sqrt(pow(x1-x2,2)+
                                  pow(y1-y2,2))

    return totaldist


## mapsolver

In [21]:
#Edges list E:
#E=[Parent in V,child in V,distance, priority(to be calculated later)

    
def mapsolver(V,E,gridsize,startnode,goalnode,iterationtimeout,Astarmode=0):
    start_time = time.time()

    # Dijkstra's
    #start and goal locations were added earlier to the nodelist, but the arguments
    #specify WHERE to start and end
    
    O_list=[] #priorityqueue
    C_list=[] #processednodes
    savedlength=0
    goalcheck=0  #used only for Dijkstra's
    iterationcounter=1
    currentnode=startnode  #starts at 'start'
    Astarcounter=0
    while True:
#         print("counter",iterationcounter)
#         print("O_list",O_list)
#         print("C_list",C_list)
#         print("")
        #if iterationcounter%1000==0:
            #print("counter",iterationcounter)
        if len(O_list)>0:
            C_list.append(O_list[0])
            O_list.pop(0)
            
            #get x and y values that we expand from
            currentnode=C_list[len(C_list)-1][1]
            #if iterationcounter>0:
                #print("trying to expand from index", C_list[len(C_list)-1][1])
                
            # if we are using dijkstras, end as soon as the goal is found     
            if Astarmode==0:
                Astarcounter+=1
                if Astarcounter==len(E):
                    break
                for j in range(len(O_list)):
                    if (O_list[j][0]== goalnode or O_list[j][1]== goalnode):
                        #print("goal reached!")
                        #print(O_list[j])
                        goalcheck=1
                        break 
            if goalcheck==1:
                break
                
            if Astarmode==1:               
                if len(C_list)==len(E):
                    break
                    
        expansioncheck=0
        for i in range(len(E)):
            duplicatecheck=0    
            if E[i][0]==currentnode: #if parent = start or last point,
                #Check that new point about to be added to 
                #O_list is not already in C_list

                for k in range(len(C_list)):
                        if E[i][0]==C_list[k][0] and E[i][1]==C_list[k][1]:
                            duplicatecheck=1
                            #print("duplicate. Point on C-list. Not added to O-list")
                            break

                for k in range(len(O_list)):
                        if E[i][0]==O_list[k][0] and E[i][1]==O_list[k][1]:
                            duplicatecheck=1
                            #print("duplicate. Point already on O-list.")
                            break

                if duplicatecheck==0:   
                    expansioncheck=1
                    O_list.append(E[i]) 
                    #if iterationcounter>130:
                        #print(O_list[len(O_list)-1])
                    #update priority of the node we just added to Olist
                    #Old length method
                    #O_list[len(O_list)-1][3]=lengthtostart(E,O_list[len(O_list)-1],startnode)
                    if iterationcounter>0:
                        
                        #lengthtostart(edgelist,matrix,gridsize,O_list,C_list,startnode,goalnode)
                        O_list[len(O_list)-1][3]=lengthtostart(E,V,gridsize,O_list,C_list,startnode,O_list[len(O_list)-1][0],Astarmode)
#                         if Astarmode==1:
#                                 #calculate heuristic value, h(n), of which I am using dist to goal, and add to priority
#                                 currentx=V[O_list[len(O_list)-1][0]][0] 
#                                 currenty=V[O_list[len(O_list)-1][0]][1]
#                                 goalx=V[goalnode][0]
#                                 goaly=V[goalnode][1]
#                                 disttogoal=math.sqrt(pow(goalx-currentx,2)+pow(goaly-currenty,2))
#                                 O_list[len(O_list)-1][3]+=disttogoal
                        
#                     else:
#                         O_list[len(O_list)-1][3]=O_list[len(O_list)-1][2]
                    
                    O_list[len(O_list)-1][3]=round(O_list[len(O_list)-1][3],2)

        #if expansioncheck==0 and iterationcounter>0:
        #    print("No expansion")
        O_list.sort(key=prioritysize)
        if iterationcounter==iterationtimeout:
            #print("max iterations")
            #print("No path found")
            elapsed_time = time.time() - start_time
            return 0,0,0,0,0
            break    
        iterationcounter+=1
    #print("O_list length",len(O_list))
    #print("")
    #print("C_list length",len(C_list))
    #print("C_list ",C_list)
    ############Begin Backtracking #############
    
    #print("backtracking for path home")

    
    pathlist=[]
    xpathlist=[]
    ypathlist=[]
    for j in range(len(O_list)):
        if (O_list[j][0]== goalnode or O_list[j][1]== goalnode):
            pathlist.append(O_list[j][1])
            pathlist.append(O_list[j][0])
            break 
    if Astarmode==1:
        goallist=[]
        for j in range(len(C_list)):
            if (C_list[j][1]== goalnode):
                goallist.append(C_list[j])
        goallist.sort(key=prioritysize)
        #print(goallist)
        pathlist.append(goallist[0][1])
        pathlist.append(goallist[0][0])
               
            
    while True:
        for i in range(len(C_list)):
            if C_list[i][1]==pathlist[len(pathlist)-1]:
                pathlist.append(C_list[i][0])
                #print(C_list[i])
                break
        if pathlist[len(pathlist)-1]==startnode:
            break
    totaldist=0
#    for g in range(len(pathlist)-1):
#             index1=pathlist[g]
#             index2=pathlist[g+1]

#             x1=V[index1][0]
#             y1=V[index1][1]
#             xpathlist.append(x1)
#             ypathlist.append(y1)
#             x2=V[index2][0]
#             y2=V[index2][1]
#            totaldist+=gridsize*2

    #print("final iteration:",iterationcounter)
    #elapsed_time = time.time() - start_time
    return pathlist #,totaldist,elapsed_time

## grid plot test

In [22]:
workspace=3
gridsize=0.5

obstacles,triangobs=defineworkspace(workspace)
sgrid=generategrid(workspace,gridsize)
filteredgrid=gridfilter(workspace,obstacles,gridsize,sgrid)
#print(len(sgrid))
#print(len(filteredgrid))

fig, ax = plt.subplots(figsize=(4,4))
ax.set_aspect('equal', adjustable='box')          
for ob in obstacles:
        rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k')
for x in range(len(filteredgrid)):
       rectangleplot(filteredgrid[x][0],filteredgrid[x][1],filteredgrid[x][2],filteredgrid[x][3],'k','b',1,0.5)   
        #xmin,xmax,ymin,ymax,Color,Color2,fill=0,Alpha=0
plt.plot(start[0],start[1],'.')
plt.plot(goal[0],goal[1],'.')   
plt.text(start[0],start[1],'start')
plt.text(goal[0],goal[1],'goal')
plt.title("Discretized 2D Space with 0.5x0.5 grid size")

i=coordinatetomatrixindex(filteredgrid,start[0],start[1])
rectangleplot(filteredgrid[i][0],filteredgrid[i][1],filteredgrid[i][2],filteredgrid[i][3],'k','y',1,1)

j=coordinatetomatrixindex(filteredgrid,goal[0],goal[1])
rectangleplot(filteredgrid[j][0],filteredgrid[j][1],filteredgrid[j][2],filteredgrid[j][3],'k','g',1,1)

plt.xlim(0,8)
plt.ylim(0,8)  
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## dijkstras w/o waypoints

In [23]:
iterationtimeout=8000

workspace=3
gridsize=0.5
Astarmode=0
if workspace==3:
    
    goal=[0.63,6.102]  
    
obstacles,triangobs=defineworkspace(workspace)
sgrid=generategrid(workspace,gridsize)
V=gridfilter(workspace,obstacles,gridsize,sgrid)

E=edgelistgenerate(workspace,gridsize,V,1)
#print("# of Edges", len(E))
#startA=start
#goalA=goal

startA=[6.8385,0.7875]
#[1.25,3.25]
goalA=[0.63,6.102]

#startA=[2.75,0.75]
#goalA=[1.25,1.25]

startnode=coordinatetomatrixindex(V,startA[0],startA[1])
goalnode=coordinatetomatrixindex(V,goalA[0],goalA[1])
#print("startnode",startnode)
#print("goalnode",goalnode)

pathlist=mapsolver(V,E,gridsize,startnode,goalnode,iterationtimeout,Astarmode)
#print(pathlist)
totaldist=0
for g in range(len(pathlist)-1):
            index1=pathlist[g]
            index2=pathlist[g+1]

            x1=(V[index1][0] +V[index1][1])/2
            y1=(V[index1][2] +V[index1][3])/2
            x2=(V[index2][0] +V[index2][1])/2
            y2=(V[index2][2] +V[index2][3])/2
            totaldist+=math.sqrt(pow(x1-x2,2)+
                                  pow(y1-y2,2))
#print("totaldist",totaldist)    

fig, ax = plt.subplots(figsize=(4,4))
ax.set_aspect('equal', adjustable='box')          
for ob in obstacles:
        rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k')
for x in range(len(V)):
       rectangleplot(V[x][0],V[x][1],V[x][2],V[x][3],'k','b',1,0.5)   
        #xmin,xmax,ymin,ymax,Color,Color2,fill=0,Alpha=0
plt.plot(start[0],start[1],'.')
plt.plot(goal[0],goal[1],'.')   
plt.text(start[0],start[1],'start')
plt.text(goal[0],goal[1],'goal')

i=coordinatetomatrixindex(V,start[0],start[1])
rectangleplot(V[i][0],V[i][1],V[i][2],V[i][3],'k','g',1,1)

j=coordinatetomatrixindex(V,goal[0],goal[1])
rectangleplot(V[j][0],V[j][1],V[j][2],V[j][3],'k','g',1,1)

for k in pathlist:
    rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1) 

plt.xlim(0,8)
plt.ylim(0,8)  
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## dijkstras with waypoints

In [24]:
workspace=3
gridsize=0.5
Astarmode=0
obstacles,triangobs=defineworkspace(workspace)
sgrid=generategrid(workspace,gridsize)
V=gridfilter(workspace,obstacles,gridsize,sgrid)
E=edgelistgenerate(workspace,gridsize,V,1)

# targetlist=[[6.8385,0.7875],[5.25,3.25],[4.25,3.25],[2.75,0.75],[1.25,0.75],
#         [1.25,2.25],[2.75,2.25],[4.25,4.25],[5.25,4.25],[5.25,5.25],
#         [4.25,5.25],[2.75,3.25],[1.25,3.25],[0.63,6.102]]  

targetlist=[[6.8385,0.7875],[5.25,3.3],[4.1,3.3],[2.75,1.1],[1.4,1.1],
        [1.4,2.25],[2.75,2.25],[3.75,4.25],[5.75,4.25],[5.75,5.4],
        [3.75,5.4],[2.4,3.25],[1.1,3.25],[0.63,6.102]]  

# targetlist=[[6.8385,0.7875],[4.75,3.25],[4.25,2.75],[2.75,0.75],[1.25,1.25],
#         [2.75,2.25],[4.75,4.25],[5.25,5.25],[1.25,3.25],[0.63,6.102]] 

targetnodelist=[]
for coord in targetlist:
    targetnodelist.append(coordinatetomatrixindex(V,coord[0],coord[1]))
print("targetnodelist",targetnodelist)

fig, ax = plt.subplots(figsize=(4,4))
ax.set_aspect('equal', adjustable='box') 
for ob in obstacles:
        rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k')
for x in range(len(V)):
       rectangleplot(V[x][0],V[x][1],V[x][2],V[x][3],'k','b',1,0.1)   
        
mainpath=[]
counter=0
for n in range(len(targetnodelist)-1):
    localstart=targetnodelist[n]
    localgoal=targetnodelist[n+1]
    pathlist=mapsolver(V,E,gridsize,localstart,localgoal,iterationtimeout,Astarmode)
    print(pathlist)
    if counter==0:
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,0.5)
    if counter==1:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','orange',1,0.5)
        
    if counter==2:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','y',1,0.5) 
    if counter==3:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','g',1,0.5) 
    if counter==4:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','g',1,0.5) 
    if counter==5:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','violet',1,0.5) 
        counter=-1        
    plt.show()    
    counter+=1  
    
    
    mainpath.extend(pathlist)
print(mainpath)



# for k in mainpath:
#     rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1) 
        #xmin,xmax,ymin,ymax,Color,Color2,fill=0,Alpha=0
plt.plot(start[0],start[1],'.')
plt.plot(goal[0],goal[1],'.')   

# plt.text(start[0],start[1],'start')
# plt.text(goal[0],goal[1],'goal')
plt.title("High-Level Path Between Waypoints")

i=coordinatetomatrixindex(V,start[0],start[1])
rectangleplot(V[i][0],V[i][1],V[i][2],V[i][3],'k','g',1,1)

j=coordinatetomatrixindex(V,goal[0],goal[1])
rectangleplot(V[j][0],V[j][1],V[j][2],V[j][3],'k','g',1,1)

#for k in targetnodelist:
#    rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1) 
    
counter=0
for coord in targetlist:
    targetnodelist.append(coordinatetomatrixindex(V,coord[0],coord[1]))
    plt.plot(coord[0],coord[1],'.',color='k')
    plt.text(coord[0],coord[1],counter,color='w')
    counter+=1
    
plt.xlim(0,8)
plt.ylim(0,8)  
plt.show()  



targetnodelist [190, 149, 122, 72, 34, 36, 74, 108, 165, 167, 110, 61, 37, 28]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[149, 148, 147, 146, 145, 159, 175, 174, 190]
[122, 135, 149]
[72, 73, 87, 88, 89, 105, 106, 122]
[34, 33, 47, 59, 71, 72]
[36, 35, 34]
[74, 60, 48, 36]
[108, 107, 106, 105, 89, 88, 74]
[165, 150, 136, 123, 108]
[167, 166, 165]
[110, 124, 137, 151, 167]
[61, 75, 90, 91, 92, 93, 94, 110]
[37, 49, 61]
[28, 27, 26, 25, 24, 23, 22, 37]
[149, 148, 147, 146, 145, 159, 175, 174, 190, 122, 135, 149, 72, 73, 87, 88, 89, 105, 106, 122, 34, 33, 47, 59, 71, 72, 36, 35, 34, 74, 60, 48, 36, 108, 107, 106, 105, 89, 88, 74, 165, 150, 136, 123, 108, 167, 166, 165, 110, 124, 137, 151, 167, 61, 75, 90, 91, 92, 93, 94, 110, 37, 49, 61, 28, 27, 26, 25, 24, 23, 22, 37]


## ---Combined planners---

## goalbiasRRTzones

In [25]:
# n =5000, r = 0.5, p goal = 0.05, and epsilon= 0.25.
#r= step distance
#p_goal= goal bias probability
#epsilon=acceptable distance to goal to end algorithm
#n=max iterations
#radius= radius to check  as path grows (creates volume)

#goalbiasRRT, but with biased sampling within a list of zones taken from 
#the high level planner


def goalbiasRRTzones(workspace,start,goal,zonelist,zonebias,
                             r,p_goal,epsilon,radius,n,plotting):
    start_time = time.time()
    #print("goal:",goal)
    obstacles,triangobs=defineworkspace(workspace)

    xylist=[]
    Treelist=[]  #indicies in xylist  [parentnode,childnode]

    xylist.append([start[0],start[1]])

    counter=0
    while True:
        biasval=random()
        #print(goalbiasval)
        if biasval<p_goal:
            #if biasval is less than 0.05
            xnode=goal[0]
            ynode=goal[1]
        
        elif p_goal<biasval and biasval<(zonebias+p_goal) :
            #randomly select a zone to randomly sample from
            
            selectedzoneindex=int(random()*len(zonelist))
            
            xnode,ynode=generate_nodezone(zonelist[selectedzoneindex][0],zonelist[selectedzoneindex][1],
                                          zonelist[selectedzoneindex][2],zonelist[selectedzoneindex][3])
        else:
            xnode,ynode=generate_node(workspace)  
            

        mindistindex=0
        mindist=99999999999
        for i in range(len(xylist)):
            totaldist=math.sqrt(((xylist[i][0]-xnode)**2)+
                                   ((xylist[i][1]-ynode)**2))  
            if totaldist<mindist:
                    mindistindex=i
                    mindist=totaldist

        xnear,ynear=nearestpoint(xylist[mindistindex][0],
                        xylist[mindistindex][1],xnode,ynode,r)

        if workspace ==0 or workspace ==1 or workspace ==2 :
            colstatus=IsSubpathCollisionFree(xylist[mindistindex][0],
                               xylist[mindistindex][1],xnear,ynear,obstacles,radius)
        if workspace ==3:
            colstatus=IsSubpathCollisionFreeV2(xylist[mindistindex][0],
                               xylist[mindistindex][1],xnear,ynear,obstacles,radius)    
            
        if colstatus==True:
            xylist.append([xnear,ynear])
            Treelist.append([mindistindex,len(xylist)-1])

        goaldist=math.sqrt(((xnear-goal[0])**2)+
                                   ((ynear-goal[1])**2))
        if goaldist<epsilon:
            #print("reached goal!")
            break
        if counter>n:
            print("counter hit maximum of",n)
            return 0,0,0
            break

        counter+=1

    #print("stopped at counter",counter)

    #parent,child

    #############Generate path####################
    pathlist=[]
    pathlist.append(Treelist[(len(Treelist)-1)])

    while True:
        for i in range(len(Treelist)):
            if Treelist[i][1]==pathlist[len(pathlist)-1][0]:
                    pathlist.append(Treelist[i])
                    break
        if pathlist[len(pathlist)-1][0]==0:
            break


        totaldist=0
        for g in range(len(pathlist)-1):
                index1=pathlist[g][0]
                index2=pathlist[g][1]
                x1=xylist[index1][0]
                y1=xylist[index1][1]
                x2=xylist[index2][0]
                y2=xylist[index2][1]
                totaldist+=math.sqrt(((x1-x2)**2)+
                                       ((y1-y2)**2))

    if plotting ==1:
        fig, ax = plt.subplots(figsize=(6,6))
        if workspace==0:
            ax.set_xlim(-1, 11)
            ax.set_ylim(-3, 3)
        if workspace==1:
            ax.set_xlim(-1, 13)
            ax.set_ylim(-1, 13)
        if workspace==2:                          
            ax.set_xlim(-6, 36)
            ax.set_ylim(-6, 6)
        for g in range(len(Treelist)):
            index1=Treelist[g][0]
            index2=Treelist[g][1]
            x1=xylist[index1][0]
            y1=xylist[index1][1]
            x2=xylist[index2][0]
            y2=xylist[index2][1]
            plt.plot([x1,x2],[y1,y2],linewidth=1)
        for g in range(len(pathlist)):
            index1=pathlist[g][0]
            index2=pathlist[g][1]
            x1=xylist[index1][0]
            y1=xylist[index1][1]
            x2=xylist[index2][0]
            y2=xylist[index2][1]
            plt.plot([x1,x2],[y1,y2],linewidth=4,color='r')
        for ob in obstacles:
                tri = Delaunay(ob)
                triangobs.append(tri)
                plt.triplot(ob[:,0], ob[:,1], tri.simplices,color='k')
                plt.plot(ob[:,0], ob[:,1],color='k')
        #plt.title("GoalBiasRRT  PathLength: "+str(totaldist))
        ax.set_aspect('equal', adjustable='box')
    
    outputpathlist=[]
    
    for g in range(len(pathlist)-1,-1,-1):
            index1=pathlist[g][0]
            index2=pathlist[g][1]
            x1=xylist[index1][0]
            y1=xylist[index1][1]
            #x2=xylist[index2][0]
            #y2=xylist[index2][1]
            outputpathlist.append([x1,y1])
            if g==0:
                x2=xylist[index2][0]
                y2=xylist[index2][1]
                outputpathlist.append([x2,y2])
            
    #elapsed_time = time.time() - start_time    
    
    return outputpathlist

## goalbiasRRT3Dzones

In [26]:
# n =5000, r = 0.5, p goal = 0.05, and epsilon= 0.25.
#n=max iterations
#r= step distance
#p_goal= goal bias probability
#epsilon=acceptable distance to goal to end algorithm

def goalbiasRRT3Dzones(workspace,start,goal,zonelist,zonebias,
                  r,p_goal,epsilon,radius,n,plotting):
    start_time = time.time()
    #print("goal:",goal)
    obstacles,triangobs=defineworkspace(workspace)

    xyzlist=[]
    Treelist=[]  #indicies in xylist  [parentnode,childnode]

    xyzlist.append([start[0],start[1],start[2]])

    counter=0
    while True:
        biasval=random()
        #print(goalbiasval)
        if biasval<p_goal:
            #if biasval is less than 0.05
            xnode=goal[0]
            ynode=goal[1]
            znode=goal[2]
        elif p_goal<biasval and biasval<(zonebias+p_goal) :
            #randomly select a zone to randomly sample from

            selectedzoneindex=int(random()*len(zonelist))

            xnode,ynode=generate_nodezone(zonelist[selectedzoneindex][0],zonelist[selectedzoneindex][1],
                                          zonelist[selectedzoneindex][2],zonelist[selectedzoneindex][3])
            #All zones are in 0.2<Z<0.75 
            zmin=0.2
            zmax=0.75
            z_range=(zmax-zmin)
            znode=np.round((random()*z_range),1)
            znode=znode+zmin
        else:
            xnode,ynode,znode=generate_node3D()      

        mindistindex=0
        mindist=99999999999
        for i in range(len(xyzlist)):
            totaldist=math.sqrt(((xyzlist[i][0]-xnode)**2)+
                                   ((xyzlist[i][1]-ynode)**2)+
                                    ((xyzlist[i][2]-znode)**2))  
            if totaldist<mindist:
                    mindistindex=i
                    mindist=totaldist

        xnear,ynear,znear=nearestpoint3D(xyzlist[mindistindex][0],
                        xyzlist[mindistindex][1],xyzlist[mindistindex][2],
                                         xnode,ynode,znode,r)
        
        colstatus=IsSubpathCollisionFree3D(xyzlist[mindistindex][0],
                               xyzlist[mindistindex][1],xyzlist[mindistindex][2],
                                         xnear,ynear,znear,obstacles,radius)
        
        if colstatus==True:
            xyzlist.append([xnear,ynear,znear])
            Treelist.append([mindistindex,len(xyzlist)-1])

        goaldist=math.sqrt(((xnear-goal[0])**2)+
                           ((ynear-goal[1])**2)+
                           ((znear-goal[2])**2))
        if goaldist<epsilon:
            #print("reached goal!")
            break
        if counter>n:
            print("counter hit maximum of",n)
            return 0,0,0
            break

        counter+=1

    #print("stopped at counter",counter)

    #parent,child


    #############Generate path####################
    pathlist=[]
    pathlist.append(Treelist[(len(Treelist)-1)])

    while True:
        for i in range(len(Treelist)):
            if Treelist[i][1]==pathlist[len(pathlist)-1][0]:
                    pathlist.append(Treelist[i])
                    break
        if pathlist[len(pathlist)-1][0]==0:
            break
            
    totaldist=0
    for g in range(len(pathlist)-1):
                index1=pathlist[g][0]
                index2=pathlist[g][1]
                x1=xyzlist[index1][0]
                y1=xyzlist[index1][1]
                z1=xyzlist[index1][1]
                x2=xyzlist[index2][0]
                y2=xyzlist[index2][1]
                z2=xyzlist[index2][2]
                totaldist+=math.sqrt(((x1-x2)**2)+
                                       ((y1-y2)**2)+
                                     ((z1-z2)**2))
   
    #elapsed_time = time.time() - start_time    
    
    if plotting ==1:
        print("local plotting actions")
      
    
    
    outputpathlist=[]
    for g in range(len(pathlist)-1,-1,-1):
                index1=pathlist[g][0]
                index2=pathlist[g][1]
                x1=xyzlist[index1][0]
                y1=xyzlist[index1][1]
                z1=xyzlist[index1][2]
                outputpathlist.append([x1,y1,z1])
                if g==0:
                    x2=xyzlist[index2][0]
                    y2=xyzlist[index2][1]
                    z2=xyzlist[index2][2]
                    outputpathlist.append([x2,y2,z2])        
            
    return outputpathlist

## high+low level planner

In [27]:
workspace=3
gridsize=0.4#0.5
Astarmode=0
obstacles,triangobs=defineworkspace(workspace)

#Syclop grid generation and edges for dijkstra's
sgrid=generategrid(workspace,gridsize)
V=gridfilter(workspace,obstacles,gridsize,sgrid)
E=edgelistgenerate(workspace,gridsize,V,1)

if workspace==3:
    start=[6.8385,0.7875]
    goal=[0.63,6.102]  
#     targetlist=[[6.8385,0.7875],[4.75,3.25],[4.25,2.75],[2.75,0.75],[1.25,1.25],
#         [2.75,2.25],[4.75,4.25],[5.25,5.25],[1.25,3.25],[0.63,6.102]] 
        
    targetlist=[[6.8385,0.7875],[5.25,3.3],[4.1,3.3],[3.75,2.25],[2.75,1.1],[1.41,1.1],
        [1.41,2.25],[2.75,2.25],[3.75,4.25],[5.75,4.25],[5.75,5.41],
        [3.75,5.41],[2.41,3.25],[1.1,3.25],[0.63,6.102]]  
if workspace==4:
    start=[6.8385,0.7875,0.25]
    goal=[0.63,6.102,0.25]
    targetlist=[[6.8385,0.7875,0.5],[6,3.3,0.5],[4,3.3,0.5],[3,1,0.5],[1.5,1,0.5],
        [1.5,2.25,0.5],[2.75,2.25,0.5],[3.75,4.5,0.5],[5.75,4.25,0.5],[6.25,5.5,0.5],
        [3.75,5.5,0.5],[2.5,3.25,0.5],[1,3.25,0.5],[0.63,6.102,0.5]]  

#convert target points to target zones     
targetnodelist=[]
for coord in targetlist:
    targetnodelist.append(coordinatetomatrixindex(V,coord[0],coord[1]))
#print("targetnodelist",targetnodelist)

fig, ax = plt.subplots(figsize=(4,4))
ax.set_aspect('equal', adjustable='box') 
for ob in obstacles:
    rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k')
for x in range(len(V)):
    rectangleplot(V[x][0],V[x][1],V[x][2],V[x][3],'k','b',1,0.3)   
        
mainpath=[] #list of V matrix indicies for overall grid path
zonelist=[] #xmin,xmax,ymin,ymax's of current path segment  to send to RRT
#[[[boundaries for segment 1 zone 1][boundaries for segment 1 zone 2]]
#[[boundaries for segment 2 zone 1][boundaries for segment 2 zone 2]]]
counter=0
for n in range(len(targetnodelist)-1):
    localstart=targetnodelist[n]
    localgoal=targetnodelist[n+1]
    pathlist=mapsolver(V,E,gridsize,localstart,localgoal,iterationtimeout,Astarmode)
    localzonelist=[] 
    #print(pathlist)
    if counter==0:
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1)
    if counter==1:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','indigo',1,1)
    if counter==2:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','y',1,1) 
    if counter==3:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','g',1,1) 
    if counter==4:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','b',1,1)  
    if counter==5:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','orange',1,1)          
    if counter==6:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','violet',1,1) 
        counter=-1        
    #plt.show()    
    counter+=1  
    mainpath.extend(pathlist)
    
    for k in pathlist:
            localzonelist.append([V[k][0],V[k][1],V[k][2],V[k][3]])
    zonelist.append(localzonelist)
    
#print(mainpath)

plt.plot(start[0],start[1],'.')
plt.plot(goal[0],goal[1],'.')   
# plt.text(start[0],start[1],'start')
# plt.text(goal[0],goal[1],'goal')

# i=coordinatetomatrixindex(V,start[0],start[1])
# rectangleplot(V[i][0],V[i][1],V[i][2],V[i][3],'k','g',1,1)

# j=coordinatetomatrixindex(V,goal[0],goal[1])
# rectangleplot(V[j][0],V[j][1],V[j][2],V[j][3],'k','g',1,1)

#for k in targetnodelist:
#    rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1) 
    
counter=0
for coord in targetlist:
    targetnodelist.append(coordinatetomatrixindex(V,coord[0],coord[1]))
    plt.plot(coord[0],coord[1],'.',color='k')
    plt.text(coord[0],coord[1],counter,color='w')
    counter+=1
    
plt.xlim(0,8)
plt.ylim(0,8) 
plt.xlabel("X")
plt.ylabel("Y")
plt.title("High-Level Path Between Waypoints")
plt.show()  



#----------------------------------------------------------------
r = 0.4 #dist between points 0.25 default
p_goal = 0.05#0.05
zonebias=0.75 #bias towards zones from high level planner in current segment
epsilon= 0.25
n =5000
workspace=3
radius=0.07

mainpath=[]

if workspace ==0 or workspace ==1 or workspace ==2 or workspace ==3:
    fig, ax = plt.subplots(figsize=(6,6))
    for g in range(len(targetlist)-1):
        #print(zonelist[g])
        #print("")
        outputpathlist=goalbiasRRTzones(
                        workspace,targetlist[g],targetlist[g+1],
                        zonelist[g],zonebias,r,p_goal,epsilon,radius,n,0)
        mainpath.extend(outputpathlist)   
        #print(g,"--",targetlist[g],targetlist[g+1])
        #print("")
        for k in zonelist[g]:
            rectangleplot(k[0],k[1],k[2],k[3],'k','violet',1,0.3) 
        
    
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                plt.plot([x1,x2],[y1,y2],linewidth=4,color='g')
    for ob in obstacles:
                rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k',1,0)    
    for n in range(len(mainpath)-1): 
                x1path=mainpath[n][0]
                y1path=mainpath[n][1]
                x2path=mainpath[n+1][0]
                y2path=mainpath[n+1][1]
                #print("line from ",x1path,y1path," to ",x2path,y2path)
                plt.plot([x1path,x2path],[y1path,y2path],linewidth=4,color='r')   
                x=x1path
                y=y1path
                xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,0.07,resolution=15,radiuslayers=3)
                plt.plot(xlist,ylist,'.')

    plt.title("GoalBias RRT With Waypoints and 75% High Level Plan Bias")
    plt.xlabel("X")
    plt.ylabel("Y")
    ax.set_aspect('equal', adjustable='box')

    
if workspace ==4:
    for g in range(len(targetlist)-1):
        outputpathlist=goalbiasRRT3D(
                        workspace,targetlist[g],targetlist[g+1],r,p_goal,epsilon,radius,n,0)
        #print(outputpathlist)
        mainpath.extend(outputpathlist) 
        
    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')
    ax.scatter(start[0],start[1],start[2])
    ax.scatter(goal[0],goal[1],goal[2],'.')   
    ax.text(start[0],start[1],start[2],'start')
    ax.text(goal[0],goal[1],goal[2],'goal')
    set_aspect_equal_3d(ax)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    
    
    colorcounter=0
    color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
           'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'b','b']
    for ob in obstacles:
            rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],
                            color[colorcounter],1,0)
            colorcounter+=1
            
    #ax.plot(outputpathlist[:,0], outputpathlist[:,1], outputpathlist[:,2]) 
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                z1=targetlist[g][2]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                z2=targetlist[g+1][2]
                plt.plot([x1,x2],[y1,y2],[z1,z2],linewidth=4,color='g')
                
    for n in range(len(mainpath)-1): 
            x1path=mainpath[n][0]
            y1path=mainpath[n][1]
            z1path=mainpath[n][2]
            x2path=mainpath[n+1][0]
            y2path=mainpath[n+1][1]
            z2path=mainpath[n+1][2]
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='b', 
                    zorder=-5)  
            x=x1path
            y=y1path
            z=z1path
            xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                          x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
            ax.plot(xlist,ylist,zlist,'.') 
            
            if n == len(outputpathlist)-1:
                x=x2path
                y=y2path
                z=z2path
                xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                              x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
                ax.plot(xlist,ylist,zlist,'.') 
       
    plt.show()   


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## high+low level planner 3D

In [28]:
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

workspace=3
gridsize=0.4
Astarmode=0
obstacles,triangobs=defineworkspace(workspace)

#Syclop grid generation and edges for dijkstra's
sgrid=generategrid(workspace,gridsize)
V=gridfilter(workspace,obstacles,gridsize,sgrid)
E=edgelistgenerate(workspace,gridsize,V,1)


if workspace==3:
    start=[6.8385,0.7875]
    goal=[0.63,6.102]  
#     targetlist=[[6.8385,0.7875],[4.75,3.25],[4.25,2.75],[2.75,0.75],[1.25,1.25],
#         [2.75,2.25],[4.75,4.25],[5.25,5.25],[1.25,3.25],[0.63,6.102]] 
        
    targetlist=[[6.8385,0.7875],[5.25,3.3],[4.1,3.3],[3.75,2.25],[2.75,1.1],[1.4,1.1],
        [1.41,2.25],[2.75,2.25],[3.75,4.25],[5.75,4.25],[5.75,5.41],
        [3.75,5.41],[2.41,3.25],[1.1,3.25],[0.63,6.102]]  
if workspace==4:
    start=[6.8385,0.7875,0.25]
    goal=[0.63,6.102,0.25]
#     targetlist=[[6.8385,0.7875,0.5],[6,3.3,0.5],[4,3.3,0.5],[3,1,0.5],[1.5,1,0.5],
#         [1.5,2.25,0.5],[2.75,2.25,0.5],[3.75,4.5,0.5],[5.75,4.25,0.5],[6.25,5.5,0.5],
#         [3.75,5.5,0.5],[2.5,3.25,0.5],[1,3.25,0.5],[0.63,6.102,0.5]]  
    
    targetlist=[[6.8385,0.7875,0.5],[5.25,3.3,0.5],[4.1,3.3,0.5],[3.75,2.25,0.5],
        [2.75,1.1,0.5],[1.4,1.1,0.5],
        [1.4,2.25,0.5],[2.75,2.25,0.5],[3.75,4.25,0.5],[5.75,4.25,0.5],[5.75,5.4,0.5],
        [3.75,5.4,0.5],[2.4,3.25,0.5],[1.1,3.25,0.5],[0.63,6.102,0.5]] 

#convert target points to target zones     
targetnodelist=[]
for coord in targetlist:
    targetnodelist.append(coordinatetomatrixindex(V,coord[0],coord[1]))
#print("targetnodelist",targetnodelist)

fig, ax = plt.subplots(figsize=(4,4))
ax.set_aspect('equal', adjustable='box') 
for ob in obstacles:
    rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k')
for x in range(len(V)):
    rectangleplot(V[x][0],V[x][1],V[x][2],V[x][3],'k','b',1,0.2)   
        
mainpath=[] #list of V matrix indicies for overall grid path
zonelist=[] #xmin,xmax,ymin,ymax's of current path segment  to send to RRT
#[[[boundaries for segment 1 zone 1][boundaries for segment 1 zone 2]]
#[[boundaries for segment 2 zone 1][boundaries for segment 2 zone 2]]]
counter=0
for n in range(len(targetnodelist)-1):
    localstart=targetnodelist[n]
    localgoal=targetnodelist[n+1]
    pathlist=mapsolver(V,E,gridsize,localstart,localgoal,iterationtimeout,Astarmode)
    localzonelist=[] 
    #print(pathlist)
    if counter==0:
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1)
    if counter==1:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','orange',1,1)
        
    if counter==2:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','y',1,1) 
    if counter==3:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','g',1,1) 
    if counter==4:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','b',1,1)
    if counter==5:   
        for k in pathlist:
            rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','violet',1,1) 
        counter=-1        
    plt.show()    
    counter+=1  
    mainpath.extend(pathlist)
    
    for k in pathlist:
            localzonelist.append([V[k][0],V[k][1],V[k][2],V[k][3]])
    zonelist.append(localzonelist)
    
#print(mainpath)

plt.plot(start[0],start[1],'.')
plt.plot(goal[0],goal[1],'.')   
plt.text(start[0],start[1],'start')
plt.text(goal[0],goal[1],'goal')

i=coordinatetomatrixindex(V,start[0],start[1])
rectangleplot(V[i][0],V[i][1],V[i][2],V[i][3],'k','g',1,1)

j=coordinatetomatrixindex(V,goal[0],goal[1])
rectangleplot(V[j][0],V[j][1],V[j][2],V[j][3],'k','g',1,1)

#for k in targetnodelist:
#    rectangleplot(V[k][0],V[k][1],V[k][2],V[k][3],'k','r',1,1) 
    
counter=0
for coord in targetlist:
    targetnodelist.append(coordinatetomatrixindex(V,coord[0],coord[1]))
    plt.plot(coord[0],coord[1],'.',color='k')
    plt.text(coord[0],coord[1],counter,color='w')
    counter+=1
    
plt.xlim(0,8)
plt.ylim(0,8)  
plt.show()  



#----------------------------------------------------------------
workspace=4
obstacles,triangobs=defineworkspace(workspace)

r = 0.4 #dist between points 0.25 default
p_goal = 0.05 #0.05
zonebias=0.75 #bias towards zones from high level planner in current segment
epsilon= 0.25
n =5000
radius=0.07

mainpath=[]

if workspace ==0 or workspace ==1 or workspace ==2 or workspace ==3:
    print("Test1")
    fig, ax = plt.subplots(figsize=(6,6))
    for g in range(len(targetlist)-1):
        outputpathlist=goalbiasRRTzones(
                        workspace,targetlist[g],targetlist[g+1],zonelist[g],zonebias,r,p_goal,epsilon,radius,n,0)
        mainpath.extend(outputpathlist)  
        for k in zonelist[g]:
            rectangleplot(k[0],k[1],k[2],k[3],'k','violet',1,0.5) 
        
    
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                plt.plot([x1,x2],[y1,y2],linewidth=4,color='g')
    for ob in obstacles:
                rectangleplot(ob[0],ob[1],ob[2],ob[3],'k','k',1,0)    
    for n in range(len(mainpath)-1): 
                x1path=mainpath[n][0]
                y1path=mainpath[n][1]
                x2path=mainpath[n+1][0]
                y2path=mainpath[n+1][1]
                #print("line from ",x1path,y1path," to ",x2path,y2path)
                plt.plot([x1path,x2path],[y1path,y2path],linewidth=4,color='r')   
                x=x1path
                y=y1path
                xlist,ylist,xouterlist,youterlist= radiusofpoints(x,y,0.07,resolution=15,radiuslayers=3)
                plt.plot(xlist,ylist,'.')

    plt.title("Intended path with waypoints")
    ax.set_aspect('equal', adjustable='box')

    
if workspace ==4:
  
    start=[6.8385,0.7875,0.25]
    goal=[0.63,6.102,0.25]
#     targetlist=[[6.8385,0.7875,0.5],[6,3.3,0.5],[4,3.3,0.5],[3,1,0.5],[1.5,1,0.5],
#         [1.5,2.25,0.5],[2.75,2.25,0.5],[3.75,4.5,0.5],[5.75,4.25,0.5],[6.25,5.5,0.5],
#         [3.75,5.5,0.5],[2.5,3.25,0.5],[1,3.25,0.5],[0.63,6.102,0.5]]  
    
    targetlist=[[6.8385,0.7875,0.5],[5.25,3.3,0.5],[4.1,3.3,0.5],[3.75,2.25,0.5],
        [2.75,1.1,0.5],[1.4,1.1,0.5],
        [1.4,2.25,0.5],[2.75,2.25,0.5],[3.75,4.25,0.5],[5.75,4.25,0.5],[5.75,5.4,0.5],
        [3.75,5.4,0.5],[2.4,3.25,0.5],[1.1,3.25,0.5],[0.63,6.102,0.5]] 
    
    
    for g in range(len(targetlist)-1):
        #print(zonelist[g])
        outputpathlist=goalbiasRRT3Dzones(
                        workspace,targetlist[g],targetlist[g+1],zonelist[g],zonebias,
                                r,p_goal,epsilon,radius,n,0)
        #print(outputpathlist)
        mainpath.extend(outputpathlist) 

    fig = plt.figure(figsize=(8,8))
    ax = fig.gca(projection='3d')
    ax.scatter(start[0],start[1],start[2])
    ax.scatter(goal[0],goal[1],goal[2],'.')   
    ax.text(start[0],start[1],start[2],'start')
    ax.text(goal[0],goal[1],goal[2],'goal')
    set_aspect_equal_3d(ax)
    ax.set_xlabel('$X$')
    ax.set_ylabel('$Y$')
    ax.set_zlabel('$Z$')
    plt.title("GoalBias RRT With Waypoints and 75% High Level Plan Bias")
    for g in range(len(targetlist)-1):
        for k in zonelist[g]:
            rectangle3Dplot(k[0],k[1],k[2],k[3],0,0,'violet',1,0.5)
    
    colorcounter=0
    color=['r','r','r','r',[0.2, 0.2, 0.2],[0.2,0.2, 0.2],[0.2, 0.2, 0.2],[0.2,0.2, 0.2],
           'b','b','g','g','b','b',[0.5, 0.5, 0.5],[0.5,0.5, 0.5],'b','b']
    for ob in obstacles:
            rectangle3Dplot(ob[0],ob[1],ob[2],ob[3],ob[4],ob[5],
                            color[colorcounter],1,0)
            colorcounter+=1
            
    #ax.plot(outputpathlist[:,0], outputpathlist[:,1], outputpathlist[:,2]) 
    for g in range(len(targetlist)-1):
                x1=targetlist[g][0]
                y1=targetlist[g][1]
                z1=targetlist[g][2]
                x2=targetlist[g+1][0]
                y2=targetlist[g+1][1]
                z2=targetlist[g+1][2]
                plt.plot([x1,x2],[y1,y2],[z1,z2],linewidth=4,color='g')
                
    for n in range(len(mainpath)-1): 
            x1path=mainpath[n][0]
            y1path=mainpath[n][1]
            z1path=mainpath[n][2]
            x2path=mainpath[n+1][0]
            y2path=mainpath[n+1][1]
            z2path=mainpath[n+1][2]
            #print("line from ",x1path,y1path," to ",x2path,y2path)
            ax.plot([x1path,x2path],[y1path,y2path],[z1path,z2path],linewidth=4,color='r', 
                    zorder=-5)  
            x=x1path
            y=y1path
            z=z1path
            xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                          x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
            ax.plot(xlist,ylist,zlist,'.') 
            
            if n == len(outputpathlist)-1:
                x=x2path
                y=y2path
                z=z2path
                xlist,ylist,zlist,xlistouter,ylistouter,zlistouter=radiusofpoints3D(
                              x,y,z,radius,perimeterresolution=5,zlayers=8,inwardlayertotal=1)
                ax.plot(xlist,ylist,zlist,'.') 
       
    plt.show()  
    
now = datetime.now()
current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)    

Current Time = 23:52:03


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Current Time = 23:52:15


In [62]:
# xlist,ylist,xouterlist,youterlist= radiusofpoints(0,0,0.07,resolution=15,radiuslayers=3)
# fig, ax = plt.subplots(figsize=(6,6))
# plt.title("Volume of points: R=0.07,3 Layers,15 points per layer")
# plt.plot(xlist,ylist,'o',color='r')
# plt.xlabel("X")
# plt.ylabel("Y")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Y')